In [ ]:
### select the SNP list which appeared in both conditions
### must satisfy the minimum read number

def read_file(in_file):
    df = pd.read_table(in_file, sep="\t")
    df['term'] = df['contig'] + ":" + df['position'].astype(str)
    return df['term'].tolist()

def select_baseline_SNPs(in_dir):
    os.chdir(in_dir)
#     f_1, f_2 = "WT.txt", "KD.txt"
    f_1, f_2 = "WT.txt", "D395A.txt"
    t1_list = read_file(f_1)
    t2_list = read_file(f_2)
    SNV_list = set(t1_list).intersection(set(t2_list))
    return SNV_list

# data_dir = "/home/galaxy/project/alleleSpecific_analysis/data/shFUS/count/Hypothesis/Fisher/heter_sites/"
# asm6A_list = select_baseline_SNPs(data_dir)
asm6A_list = select_baseline_SNPs(data_dir)

print(len(asm6A_list))

In [ ]:
# ### filter > IP/INPUT >= 1,2 or 3;
# os.chdir("/home/galaxy/project/alleleSpecific_analysis/data/mouse_ASm6A/count/Hypothesis/comparision/ASm6A/")
# # os.chdir("/home/galaxy/project/alleleSpecific_analysis/data/shFUS/count/Hypothesis/comparision/ASm6A/")

# def filter_file(in_file):
#     df = pd.read_table(in_file)
#     df.columns = ['chr','s','e','info','ref','alt']
#     df['mark'] = df.iloc[:,3].str.split(";").str[2]
#     ## IP/Input > 2
#     df_res = df[((df['mark']=="ref") & (df['ref']>=2)) | ((df['mark']=="alt") & (df['alt']>=2))]
#     del df_res['mark']
#     return df_res

# result_dir = "filter/"
# os.system("mkdir -p %s" % result_dir)

# # nc = "WT.bed"
# # kd = "KD.bed"
# nc = "WT.bed"
# kd = "D395A.bed"
# df_nc, df_kd = filter_file(nc), filter_file(kd)
# df_nc.to_csv("filter/%s"%nc, sep="\t", index=False, header=False)
# df_kd.to_csv("filter/%s"%kd, sep="\t", index=False, header=False)

In [ ]:
####################################### different filter methods ####################################
######### filter: within m6A peaks
wt_peak = "/home/galaxy/project/alleleSpecific_analysis/data/shFUS/peak/rm_m6Am/WT_m6A.bed"
kd_peak = "/home/galaxy/project/alleleSpecific_analysis/data/shFUS/peak/rm_m6Am/KD_m6A.bed"

####### filter: select high 
result_dir = "highFC/"
os.system("mkdir -p %s" % result_dir)

def filter_file(in_file):
    res = "%s/%s"% (result_dir, in_file)
    df = pd.read_table(in_file)
    df = df[((df.iloc[:,5]/df.iloc[:,4])>2) | ((df.iloc[:,5]/df.iloc[:,4])<0.5)]
    df.to_csv("highFC/%s"%in_file, sep="\t", index=False, header=False)
#     return df
# filter_file("Nc1.bed")
# filter_file("Fus2.bed")
filter_file("WT.bed")
filter_file("D395A.bed")

In [ ]:
#### stat ASm6A change
os.chdir("/home/galaxy/project/alleleSpecific_analysis/data/shFUS/count/Hypothesis/comparision/ASm6A/filter/withinm6A/")
nc = "WT.bed"
kd = "KD.bed"

def get_termlist(in_file):
    df = pd.read_table(in_file)
    df['term'] = df.iloc[:,0] + ":" + df.iloc[:,2].astype(str)
    return df['term'].tolist()

nc_list, kd_list = get_termlist(nc), get_termlist(kd)
print("seperately: ", len(nc_list), len(kd_list))
nc_list = set(nc_list).intersection(set(asm6A_list))
kd_list = set(kd_list).intersection(set(asm6A_list))
print("based on common heter: ", len(nc_list), len(kd_list))
print("#########################################")
yes_to_no = set(nc_list).difference(set(kd_list))
print("yes_to_no: %d" % len(yes_to_no))
with open("yes_to_no.txt", 'w') as fw:
    fw.writelines(["%s\n"%x for x in list(yes_to_no)])
###
no_to_yes = set(kd_list).difference(set(nc_list))
print("no_to_yes: %d" % len(no_to_yes))
###
still_yes = set(nc_list).intersection(set(kd_list))
print("still yes: %d" % len(still_yes))

### stat the proportion of no_to_yes to most-condifent ASm6A sites
# # base_dir = "/home/galaxy/project/alleleSpecific_analysis/data/shFUS/count/read_matrix/most_confident/"
# # wt_list = pd.read_table("%s/Nc1_highConf.txt"%base_dir)['ID'].tolist()
# # kd_list = pd.read_table("%s/Fus2_highConf.txt"%base_dir)['ID'].tolist()
# #
# high_yes_to_no = set(yes_to_no).intersection(set(wt_list))
# high_prop = len(high_yes_to_no)/len(yes_to_no)
# print(len(high_yes_to_no), high_prop)
# #
# high_no_to_yes = set(no_to_yes).intersection(set(kd_list))
# high_prop = len(high_no_to_yes)/len(no_to_yes)
# print(len(high_no_to_yes), high_prop)

In [ ]:
# still yes
nc = "WT.bed"
kd = "KD.bed"

def read_bed(in_file, data_type):
    df = pd.read_table(in_file)
    df['AR_%s'%data_type] = df.iloc[:,3].str.split(";").str[1].astype(float)
    df['term'] = df.iloc[:,0] + ":" + df.iloc[:,2].astype(str)
    return df[['term', 'AR_%s'%data_type]]
    
def extract_ar():
    df_nc, df_kd = read_bed(nc, "nc"), read_bed(kd, "kd")
    df_nc = df_nc[df_nc['term'].isin(still_yes)]
    df_kd = df_kd[df_kd['term'].isin(still_yes)]
    df_t = df_nc.merge(df_kd, on="term")
    return df_t

df = extract_ar()
df['diff'] = df['AR_nc'] - df['AR_kd']
a = df[(df['AR_nc']>0.5) & (df['AR_kd']<0.5)]
b = df[(df['AR_nc']<0.5) & (df['AR_kd']>0.5)]
print(len(a), len(b), len(still_yes)-len(a)-len(b))
#
c = df[(df['AR_nc']>0.5) & (df['AR_kd']>0.5)]
d = df[(df['AR_nc']<0.5) & (df['AR_kd']<0.5)]
cd = pd.concat([c,d])
cd.to_csv("unchanged_ASm6A.bed", sep="\t", index=False)
# print(len(df[df['diff']>0]), len(df[df['diff']<=0]))

In [ ]:
## merge wt and kd

os.chdir("/home/galaxy/project/alleleSpecific_analysis/data/shFUS/count/new_count/sig/Fisher/")
wt, kd = "Nc1.txt", "Fus2.txt"

def read_file(in_file, data_type):
    df = pd.read_table(in_file)
    df = df[["contig","position","refRPKM_ratio", "altRPKM_ratio", "allelicRatio"]]
    df.columns = ["contig","position","refRPKM_%s"%data_type, "altRPKM_%s"%data_type, "AR_%s"%data_type]
    return df
df_wt, df_kd = read_file(wt, "wt"), read_file(kd, "kd")
df = df_wt.merge(df_kd, on=["contig","position"])
df['term'] = df.iloc[:,0] + ":" + df.iloc[:,1].astype(str)
df = df.round(2)
res = "total.counts"
df.to_csv(res, sep="\t", index=False)

In [ ]:
### 
os.chdir("/home/galaxy/project/alleleSpecific_analysis/data/shFUS/count/new_count/sig/Fisher/")
total = "total.counts"
df = pd.read_table(total)
df_res = df[df['term'].isin(no_to_yes)]
res = "no_to_yes.txt"
# df_res = df[df['term'].isin(yes_to_no)]
# res = "yes_to_no.txt"
df_res.to_csv(res, sep="\t", index=False)

In [ ]:
### compare the read count
# os.chdir("/home/galaxy/project/alleleSpecific_analysis/data/shFUS/count/sig/comparision/ASE/")
# ase1 = "Nc1.bed"
# ase2 = "Fus2.bed"
# os.chdir("/home/galaxy/project/alleleSpecific_analysis/data/shFUS/count/")
# txt_list = glob.glob("*.txt")
# #
# name_dict = {"Fus2-inp_L4_710D01.readcounts.txt":"Fus2-input", "Fus2-ip_L4_712D01.readcounts.txt":"Fus2-ip", "Nc1-inp_L4_709D01.readcounts.txt":"Nc1-input", "Nc1-ip_L4_711D01.readcounts.txt":"Nc1-ip"}
# ###
# df_total = pd.DataFrame()
# ###
# df = pd.read_table(txt_list[0])
# df = df[['contig','position','refCount','altCount']]
# p = name_dict[txt_list[0]]
# df.columns = ['contig','position','refCount_%s'%p,'altCount_%s'%p]
# df_total = df
# ##################
# for txt in txt_list[1:]:
#     print(txt)
#     df = pd.read_table(txt)
#     df = df[['contig','position','refCount','altCount']]
#     p = name_dict[txt]
#     df.columns = ['contig','position','refCount_%s'%p,'altCount_%s'%p]
#     df_total = df_total.merge(df)
# df_total['term'] = df_total['contig'] + ":" + df_total['position'].astype(str)
# df_total.to_csv("total.readcounts.txt", sep="\t", index=False)

In [ ]:
def read_file(in_file):
    df = pd.read_table(in_file, sep="\t")
    ### qvalue < 0.05
    df = df[df['qvalue'] < 0.05]
    ###############################
    df['term'] = df['contig'] + ":" + df['position'].astype(str)
    term_list = df['term'].tolist()
    # term_list = [x for x in term_list if x in SNV_list]
    #####
    df['mark'] = np.where(df['allelicRatio']>0.5, "alt", "ref")
    mark_list = df['mark'].tolist()
    return term_list, mark_list

def stat_number():
    t1_list, m1_list = read_file(f_1)
    t2_list, m2_list = read_file(f_2)
    print(len(set(t1_list)), len(set(t2_list)))
    yes_to_no = set(t1_list).difference(set(t2_list))
    yes_still_yes = set(t1_list).intersection(set(t2_list))
    no_to_yes = set(t2_list).difference(set(t1_list))
    print(len(yes_to_no), len(yes_still_yes), len(no_to_yes))
    return list(set(t1_list)), list(set(yes_to_no)), list(set(yes_still_yes)), list(set(no_to_yes))
    
### ASm6A change
os.chdir("/home/galaxy/project/alleleSpecific_analysis/data/shFUS/count/sig/Fisher/")
f_1, f_2 = "Nc1.txt", "Fus2.txt"
m6a_0, m6a_1, m6a_2, m6a_3 = stat_number()
print("###################################")
#### ASE change
# os.chdir("/home/galaxy/project/alleleSpecific_analysis/data/mouse_ASm6A/MeRIP/sig/binomi/")
os.chdir("/home/galaxy/project/alleleSpecific_analysis/data/shFUS/count/sig/binomi/")
f_1, f_2 = "Nc1-inp.txt", "Fus2-inp.txt"
ase_0, ase_1, ase_2, ase_3 = stat_number()
print("###################################")
overlap_0 = set(m6a_0).intersection(set(ase_0))
m6a_loss = [x for x in m6a_1 if x in overlap_0]
m6a_ase_loss = set(m6a_loss).intersection(set(ase_1))
print(len(overlap_0), len(m6a_loss), len(m6a_ase_loss))
#m6a_ase_overlap = set(m6a_1).intersection(set(ase_1))
#print(len(overlap_0), len(m6a_ase_overlap))

In [ ]:
### pick out oppo direction
contig_list = [x.split(":")[0] for x in m6a_ase_loss]
position = [int(x.split(":")[1]) for x in m6a_ase_loss]
df_target = pd.DataFrame({"contig": contig_list, "position": position})
# print(df_target.head())
###
asm6a = "/home/galaxy/project/alleleSpecific_analysis/data/shFUS/count/sig/Fisher/Nc1.txt"
ase = "/home/galaxy/project/alleleSpecific_analysis/data/shFUS/count/sig/binomi/Nc1-inp.txt"

def overlap_and_mark(in_file, data_type):
    df = pd.read_table(in_file, sep="\t")
    df = df[df['qvalue'] < 0.05]
    df = df.merge(df_target, on=['contig', 'position'], how="right").dropna()
    df['mark'] = np.where(df['allelicRatio']>0.5, "alt", "ref")
    df['start'] = df['position'] - 1
    df = df[['contig', 'start', 'position', 'refAllele', 'altAllele', 'mark']]
    df.columns = ['contig', 'start', 'position', 'refAllele', 'altAllele', 'mark_%s'%data_type]
    return df

df_asm6a = overlap_and_mark(asm6a, "asm6a")
df_ase = overlap_and_mark(ase, "ase")
#
df_raw = df_asm6a.merge(df_ase, on=['contig', 'start', 'position']).dropna()
print(len(df_raw))
# print(df_raw.head())
df_res = df_raw[df_raw['mark_asm6a'] != df_raw['mark_ase']]
print(len(df_res))
res = "/home/galaxy/project/alleleSpecific_analysis/data/shFUS/count/sig/oppo_direction/results.bed2"
# df_res.to_csv(res, sep="\t", index=False)  # header=False, 

#### Trending larger or smaller

In [1]:
os.chdir("/home/galaxy/project/alleleSpecific_analysis/data/shFUS/count/Hypothesis/comparision/ASm6A/filter/withinm6A/")
kd, wt = "KD.bed", "WT.bed"
df_kd, df_wt = pd.read_table(kd, header=None), pd.read_table(wt, header=None)
print(df_kd.head())
df_wt.head()

   chr10  122668295  122668296  G>A;0.08;alt  0.48   5.68
0  chr10   46957455   46957456  G>A;0.42;alt  5.11   7.04
1  chr10   64564891   64564892  G>T;0.29;alt  1.01   2.42
2   chr1  107600578  107600579  C>T;0.58;ref  7.32   5.31
3   chr1  110753764  110753765  G>C;0.26;alt  4.30  12.51
4  chr11   10800531   10800532  G>A;0.36;alt  3.32   5.98


,chr10,6275222,6275223,G>A;0.26;alt,2.75,7.93
0,chr10,63958111,63958112,T>C;0.03;alt,0.46,13.31
1,chr10,64966931,64966932,G>A;1.0;ref,11.81,0.00
2,chr1,109637228,109637229,C>G;0.66;ref,9.06,4.77
3,chr11,108380381,108380382,G>A;1.0;ref,18.66,0.00
4,chr11,108384206,108384207,C>T;0.78;ref,5.64,1.56
